In [18]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# LeNet-5

### Input Shape: 28x28x1(MNIST)
**Convolution Layers**

1. ConV_1 (5x5, s=1, activation='tanh', filters=  6, padding='same')
2. Pooling_2 (f=2, s=2, padding='valid')
3. ConV_3 (5x5, s=1, activation='tanh', filters= 16, padding='valid')
4. Pooling_4 (f=2, s=2, padding='valid')
5. ConV_5 (5x5, s=1, activation='tanh', filters=120, padding='valid')

**Fully Connected Layers**

6. Dense(units=84, activation='tanh')

**Outputs**

7. Dense(units=10, activation='softmax')

## Build Models

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

In [13]:
def build_lenet_5(input_shape, num_classes=10):
    
    inputs = keras.Input(shape = input_shape)

    conv1 = Conv2D(kernel_size=5,
                   strides=1,
                   activation='tanh',
                   filters=6,
                   padding='same')(inputs)
    pool2 = AveragePooling2D(pool_size=2,
                             strides=2,
                             padding='valid')(conv1)
    conv3 = Conv2D(kernel_size=5,
                   strides=1,
                   activation='tanh',
                   filters=16,
                   padding='valid')(pool2)
    pool4 = AveragePooling2D(pool_size=2,
                             strides=2,
                             padding='valid')(conv3)
    conv5 = Conv2D(kernel_size=5,
                   strides=1,
                   activation='tanh',
                   filters=120,
                   padding='valid')(pool4)
    
    flat = Flatten()(conv5)
    
    dense6 = Dense(units=84,
                   activation='tanh')(flat)
    
    outputs = Dense(units=num_classes,
                    activation='softmax')(dense6)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='LeNet_5')
    
    return model

In [14]:
model = build_lenet_5(input_shape=(28,28,1))
model.summary()

Model: "LeNet_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_3 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_4 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_6 (Conv2D)           (None, 1, 1, 120)         4812

In [15]:
def lr_schedule(epoch):
    if epoch <= 2:
        lr = 5e-4
    elif epoch > 2 and epoch <= 5:
        lr = 2e-4
    elif epoch >5 and epoch <= 9:
        lr = 5e-5
    else:
        lr = 1e-5
    return lr

In [17]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

lr_scheduler = LearningRateScheduler(lr_schedule)
checkpoints = ModelCheckpoint(filepath='path_to_svae_file/file.hdf5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True
                             )
callbacks = [checkpoints, lr_scheduler]
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy']
             )

In [ ]:
hist = model.fit()